In [ ]:
"""
To look at the status of our limb processing
"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload
import numpy_utils as nu
import trimesh_utils as tu
import error_detection as ed
import networkx as nx
import networkx_utils as xu
import neuron_utils as nru
import neuron_visualizations as nviz

INFO - 2020-12-09 17:39:12,464 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-12-09 17:39:12,465 - settings - Setting database.user to celiib
INFO - 2020-12-09 17:39:12,466 - settings - Setting database.password to newceliipass
INFO - 2020-12-09 17:39:12,470 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-12-09 17:39:12,471 - settings - Setting enable_python_native_blobs to True
INFO - 2020-12-09 17:39:12,481 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-12-09 17:39:12,690 - settings - Setting enable_python_native_blobs to True


In [2]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 30)
print(f"Sleeping {random_sleep_sec} sec before conneting")
#time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-12-09 17:39:12,722 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-12-09 17:39:12,722 - settings - Setting database.user to celiib
INFO - 2020-12-09 17:39:12,723 - settings - Setting database.password to newceliipass
INFO - 2020-12-09 17:39:12,725 - settings - Setting enable_python_native_blobs to True


Sleeping 7 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2020-12-09 17:39:12,925 - settings - Setting enable_python_native_blobs to True


In [4]:
len(minnie.LimbStats())

9059

In [12]:
len(minnie.Decomposition.proj() & minnie.LimbStats.proj())

17073

In [13]:
(minnie.schema.jobs & "table_name='__limb_stats'")#.delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__limb_stats,34e8ab027e7dcce86fb9d1d630f7f6e1,error,=BLOB=,"Exception: neuron 539775 did not fix the manifold edges (meshlab script failed) with output:QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root' Current Plugins Dir is: /meshlab/src/distrib/plugins Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that does not correspond to any dll or javascript code; please delete all the spurious xml files Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that does not correspond to any dll or javascript code; please delete all the spurious xml files Opening a file with extention off FilterScript Reading filter with name Remove Duplicate Vertices Reading filter with name Smooths normals on a point sets Reading Param with name K : RichInt Reading Param with name useDist : RichBool Reading filter with name Surface Reconstruction: Screened Poisson Reading filter with name Delete Current Mesh Loading Plugins: Total 104 filtering actions Total 1 io plugins Mesh /notebooks/Neurosignal_Final/Poisson_temp/neuron_539775.off loaded has 18826 vn 37432 fn output mesh /notebooks/Neurosignal_Final/Poisson_temp/neuron_539775_poisson.off Apply FilterScript: '/notebooks/Neurosignal_Final/Poisson_temp/poisson_570582.mls' Starting Script of 4 actionsfilter: Remove Duplicate Vertices no additional memory available!!! memory required: 901008 LOG: 2 Removed 0 duplicated vertices Removed 0 duplicated vertices filter: Smooths normals on a point sets no additional memory available!!! memory required: 901008 Removed 0 duplicated vertices filter: Surface Reconstruction: Screened Poisson no additional memory available!!! memory required: 901008 Removed 0 duplicated vertices Problem with filter: Surface Reconstruction: Screened Poisson Failed to apply script file /notebooks/Neurosignal_Final/Poisson_temp/poisson_570582.mls",=BLOB=,celiib@10.28.0.140,at-node13,29,124981,2020-12-09 14:07:34
__limb_stats,353f7f1eef46b73597781df5afd54ae5,error,=BLOB=,"Exception: neuron 211624 did not fix the manifold edges (meshlab script failed) with output:QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root' Current Plugins Dir is: /meshlab/src/distrib/plugins Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that does not correspond to any dll or javascript code; please delete all the spurious xml files Error While parsing the XML filter plugin descriptors: We are trying to load a xml file that does not correspond to any dll or javascript code; please delete all the spurious xml files Opening a file with extention off FilterScript Reading filter with name Remove Duplicate Vertices Reading filter with name Smooths normals on a point sets Reading Param with name K : RichInt Reading Param with name useDist : RichBool Reading filter with name Surface Reconstruction: Screened Poisson Reading filter with name Delete Current Mesh Loading Plugins: Total 104 filtering actions Total 1 io plugins Mesh /notebooks/Neurosignal_Final/Poisson_temp/neuron_211624.off loaded has 15280 vn 29986 fn output mesh /notebooks/Neurosignal_Final/Poisson_temp/neuron_211624_poisson.off Apply FilterScript: '/notebooks/Neurosignal_Final/Poisson_temp/poisson_850543.mls' Starting Script of 4 actionsfilter: Remove Duplicate Vertices no additional memory available!!! memory required: 726552 LOG: 2 Removed 0 duplicated vertices Removed 0 duplicated vertices filter: Smooths normals on a point sets no additional memory available!!! memory required: 726552 Removed 0 duplicated vertices filter: Surface Reconstruction: Screened Poi

# Downloading the Data

In [15]:
import time
download_start_time = time.time()
total_data_dict = minnie.LimbStats.fetch(as_dict=True)
print(f"Total time for download = {time.time() - download_start_time}")

Total time for download = 27.944189310073853


# Convert to a pandas table

In [17]:
import pandas as pd
import numpy as np
path_df = pd.DataFrame.from_dict(total_data_dict)

In [20]:
path_df.to_pickle("neuron_dendrite_path_data")
df_recov = pd.read_pickle("neuron_dendrite_path_data")
df_recov

,segment_id,decimation_version,decimation_ratio,limb_idx,path_idx,soma_angle,n0_width_median_mesh_center,n0_width_no_spine_median_mesh_center,n0_n_spines,n0_total_spine_volume,...,n0_sibling_angle,n1_width_median_mesh_center,n1_width_no_spine_median_mesh_center,n1_n_spines,n1_total_spine_volume,n1_spine_volume_median,n1_spine_volume_density,n1_skeletal_length,n1_parent_angle,n1_sibling_angle
0,864691134884740858,0,0.25,0,0,155.81,192.16,192.16,0,0.000000e+00,...,140.83,328.27,327.31,4,1.846222e+08,50054356.20,2612.59,70666.48,13.53,96.80
1,864691134884740858,0,0.25,0,1,155.81,192.16,192.16,0,0.000000e+00,...,140.83,99.67,99.67,0,0.000000e+00,0.00,0.00,12721.84,84.41,96.80
2,864691134884740858,0,0.25,0,2,155.81,83.79,83.79,0,0.000000e+00,...,140.83,86.33,86.33,0,0.000000e+00,0.00,0.00,5910.65,29.57,31.68
3,864691134884740858,0,0.25,0,3,155.81,83.79,83.79,0,0.000000e+00,...,140.83,63.25,63.25,0,0.000000e+00,0.00,0.00,6464.83,4.32,31.68
4,864691134884740858,0,0.25,1,0,36.37,363.80,363.80,0,0.000000e+00,...,54.68,397.66,317.97,44,2.416440e+09,42964302.31,46091.42,52427.12,18.83,52.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278313,864691137196931137,0,0.25,3,1,156.29,461.64,461.64,0,0.000000e+00,...,114.81,288.77,288.77,6,3.020934e+08,49298399.24,13474.30,22419.97,86.21,99.05
278314,864691137196931137,0,0.25,3,2,156.29,396.01,396.01,2,1.380452e+08,...,114.81,333.88,284.49,25,2.068806e+09,50627595.27,37042.28,55849.85,3.90,58.52
278315,864691137196931137,0,0.25,3,3,156.29,396.01,396.01,2,1.380452e+08,...,114.81,320.84,280.30,26,1.225729e+09,19951175.90,16039.65,76418.71,60.77,58.52
278316,864691137196931137,0,0.25,4,0,105.23,327.06,313.81,23,1.596199e+09,...,61.65,87.42,87.42,0,0.000000e+00,0.00,0.00,5459.70,102.40,111.85


# Saving the Data

In [ ]:
import system_utils as su
su.decompress_pickle(total_data_dict,"")

# Seeing If Can Get approximately labeled data by the angle (spot checking neurons)

In [54]:
angle_interval = [164,180]
restrict_table = (minnie.LimbStats() & f"soma_angle>={angle_interval[0]} AND soma_angle<={angle_interval[-1]}")
print(f"Length of restricted table = {len(restrict_table)}")
curr_data = restrict_table.fetch("segment_id","limb_idx")

Length of restricted table = 9816


In [55]:
restrict_table

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),limb_idx the limb id path was taken from,path_idx path identifier,soma_angle,n0_width_median_mesh_center,n0_width_no_spine_median_mesh_center,n0_n_spines,n0_total_spine_volume,n0_spine_volume_median,n0_spine_volume_density,n0_skeletal_length,n0_parent_angle,n0_sibling_angle,n1_width_median_mesh_center,n1_width_no_spine_median_mesh_center,n1_n_spines,n1_total_spine_volume,n1_spine_volume_median,n1_spine_volume_density,n1_skeletal_length,n1_parent_angle,n1_sibling_angle
864691134884741370,0,0.25,4,0,167.39,67.3,67.3,0,0.0,0.0,0.0,7458.22,87.81,99.48,55.26,55.26,0,0.0,0.0,0.0,8338.27,134.35,146.58
864691134884741370,0,0.25,4,1,167.39,67.3,67.3,0,0.0,0.0,0.0,7458.22,87.81,99.48,59.43,59.43,0,0.0,0.0,0.0,11318.9,22.68,146.58
864691134884748026,0,0.25,8,0,173.77,195.35,195.35,0,0.0,0.0,0.0,1409.29,11.65,168.27,113.31,113.31,0,0.0,0.0,0.0,34587.29,14.16,40.68
864691134884748026,0,0.25,8,1,173.77,195.35,195.35,0,0.0,0.0,0.0,1409.29,11.65,168.27,86.31,86.31,0,0.0,0.0,0.0,6643.4,27.15,40.68
864691134884748282,0,0.25,4,0,172.39,320.71,320.71,2,62458599.48,31229299.74,24469.4,2552.52,28.1,113.94,84.04,84.04,0,0.0,0.0,0.0,11723.29,26.39,66.62
864691134884748282,0,0.25,4,1,172.39,320.71,320.71,2,62458599.48,31229299.74,24469.4,2552.52,28.1,113.94,192.67,192.67,0,0.0,0.0,0.0,22178.18,44.42,66.62
864691134884749562,0,0.25,0,0,169.02,142.36,142.36,0,0.0,0.0,0.0,8504.98,92.99,132.06,150.39,150.39,0,0.0,0.0,0.0,3957.09,14.86,80.96
864691134884749562,0,0.25,0,1,169.02,142.36,142.36,0,0.0,0.0,0.0,8504.98,92.99,132.06,96.31,96.31,0,0.0,0.0,0.0,39344.17,94.54,80.96
864691134884754426,0,0.25,0,0,165.45,428.95,428.95,0,0.0,0.0,0.0,4058.08,112.94,133.53,290.96,290.96,0,0.0,0.0,0.0,4367.09,59.14,59.23
864691134884754426,0,0.25,0,1,165.45,428.95,428.95,0,0.0,0.0,0.0,4058.08,112.94,133.53,400.9,400.9,3,166946018.43,78035612.05,16855.6,9904.48,11.8,59.23


In [56]:
unique_seg_id_branch_idx = np.unique(np.array(curr_data).T,axis=0)
unique_seg_id_branch_idx

array([[864691134884741370,                  4],
       [864691134884748026,                  8],
       [864691134884748282,                  4],
       ...,
       [864691137196899649,                  7],
       [864691137196912449,                  0],
       [864691137196912449,                  1]])

In [35]:
correct_axon_classification = []
wrong_axon_classification = []

In [57]:
correct_axon_classification

[[864691134884740858, 0], [864691134884741114, 1], [864691134884741370, 4]]

# Looking at an example branch

In [76]:
idx = 6
curr_segment_id,curr_limb_idx = unique_seg_id_branch_idx[idx]
curr_segment_id,curr_limb_idx

(864691134884767226, 4)

In [77]:
import neuron_visualizations as nviz
neuron_obj = (minnie.Decomposition & dict(segment_id=curr_segment_id)).fetch1("decomposition")

Decompressing Neuron in minimal output mode...please wait


In [78]:
nviz = reload(nviz)
print(f"Segment_id:{curr_segment_id}, Limb: {curr_limb_idx}")
nviz.visualize_neuron(neuron_obj,
                     visualize_type=["mesh"],
                     limb_branch_dict={f"L{curr_limb_idx}":"all"},
                     mesh_resolution="limb",
                     mesh_color="red",
                      mesh_color_alpha=1,
                     mesh_whole_neuron=True,
                     verbose=False)

Segment_id:864691134884767226, Limb: 4


In [69]:
correct_axon_classification.append([curr_segment_id,curr_limb_idx])

In [ ]:
wrong_axon_classification.append([curr_segment_id,curr_limb_idx])